<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. dash table 相關

#### 1-1. 篩選

In [ ]:
app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",  # 啟用內建篩選
        fixed_rows={'headers': True}  # 固定標題
    )
])

#### 1-2. 固定標題

是的！如果 dash_table.DataTable 的欄位很多，會自動出現水平捲動條，你可以使用 fixed_rows={'headers': True} 來固定標題列，並搭配 style_table 設定 overflowX="auto" 來確保表格可以水平滾動。

In [ ]:
import dash
from dash import dcc, html, dash_table
import pandas as pd

app = dash.Dash(__name__)

# 產生多欄位的範例數據
df = pd.DataFrame({
    "姓名": ["Alice", "Bob", "Charlie", "David"],
    "年齡": [25, 30, 35, 40],
    "城市": ["台北", "台中", "高雄", "新北"],
    "職業": ["工程師", "設計師", "醫生", "老師"],
    "公司": ["公司A", "公司B", "公司C", "公司D"],
    "興趣": ["閱讀", "運動", "音樂", "電影"],
    "國籍": ["台灣", "美國", "日本", "德國"],
    "婚姻狀態": ["未婚", "已婚", "未婚", "已婚"],
    "收入": [50000, 60000, 70000, 80000],
    "語言": ["中文", "英文", "日文", "德文"]
})

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),

        # 啟用篩選功能
        filter_action="native",

        # 啟用排序功能
        sort_action="native",

        # 固定標題列
        fixed_rows={'headers': True},

        # 設定表格最大高度，讓標題可見 (允許水平捲動：style_table={'overflowX': 'auto'})
        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},

        # 設定標題列樣式
        style_header={
            'backgroundColor': 'lightgrey',
            'fontWeight': 'bold',
            'textAlign': 'center'
        },

        # 設定內容樣式 (調整欄寬，避免欄位過寬或過窄)
        style_cell={
            'minWidth': '100px', 'maxWidth': '200px', 'whiteSpace': 'normal',
            'textAlign': 'left'
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


## 2. 圖片

#### 2-1. flask 圖片

In [ ]:
import os
from flask import Flask, send_from_directory
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

# ✅ 圖片資料夾的根目錄（請修改為你的實際路徑）
IMAGE_ROOT = os.path.abspath("../../../data_source")

# ✅ 初始化 Dash（使用 Flask 提供靜態檔案服務）
server = Flask(__name__)
app = dash.Dash(__name__, server=server)

# ✅ Flask 端點，提供圖片
@server.route("/images/<path:image_path>")
def serve_image(image_path):
    """提供靜態圖片"""
    directory, filename = os.path.split(image_path)
    return send_from_directory(directory, filename)

# ✅ 範例表格數據
df = [
    {"row_index": 1, "date": "20240210", "base": "A", "STAT": "X", "chart_name": "chart1", "time": "1200"},
    {"row_index": 2, "date": "20240210", "base": "A", "STAT": "X", "chart_name": "chart1", "time": "1200"},
    {"row_index": 3, "date": "20240210", "base": "A", "STAT": "X", "chart_name": "chart1", "time": "1200"},
    {"row_index": 4, "date": "20240211", "base": "B", "STAT": "Y", "chart_name": "chart2", "time": "1300"},
    {"row_index": 5, "date": "20240212", "base": "C", "STAT": "Z", "chart_name": "chart3", "time": "1400"},
    {"row_index": 6, "date": "20240212", "base": "C", "STAT": "Z", "chart_name": "chart3", "time": "1400"},
    {"row_index": 7, "date": "20240212", "base": "C", "STAT": "Z", "chart_name": "chart3", "time": "1400"},
    {"row_index": 8, "date": "20240212", "base": "C", "STAT": "Z", "chart_name": "chart3", "time": "1400"},
]

# ✅ Dash Layout
app.layout = html.Div([
    html.H1("Dash Table + 自動顯示符合條件的圖片"),

    # ✅ Table 顯示所有 row
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df[0].keys()],
        data=df,
        style_table={'overflowX': 'auto', 'height': '400px'},
        fixed_rows={'headers': True}
    ),

    html.H3("對應的圖片："),

    # ✅ 這裡放所有符合條件的圖片
    html.Div(id="figure_div", style={"display": "flex", "flex-wrap": "wrap"}) # 確保圖片可以自動換行
])

# ✅ 動態生成符合條件的圖片路徑
def get_image_path(row):
    """根據 row 內容計算對應的圖片路徑"""
    image_filename = f"{row['chart_name']}__{row['base']}__{row['time']}.png"
    image_folder = os.path.join(IMAGE_ROOT, row["date"], row["base"], "figure", row["STAT"])
    image_path = os.path.join(image_folder, image_filename)

    # 確保圖片存在
    if os.path.exists(image_path):
        return image_filename, f"/images/{image_path}"
    else:
        return None, None  # 若找不到圖片則回傳 None

# ✅ 計算 Table 內所有符合條件的圖片，並顯示在 `figure_div`
@app.callback(
    Output("figure_div", "children"),
    Input("table", "data")
)
def update_images(table_data):
    if not table_data:
        return html.Div("無資料")

    # 存放已經顯示過的圖片，避免重複顯示
    unique_images = set()

    image_elements = []
    for row in table_data:
        image_filename, image_src = get_image_path(row)
        if image_src and image_src not in unique_images:
            unique_images.add(image_src)
            image_elements.append(html.Div([
                html.Img(src=image_src, style={"width": "200px", "margin": "10px"}),
                html.P(f"Image Filename: {image_filename}")
            ]))

    if not image_elements:
        return html.Div("沒有找到符合條件的圖片")

    return image_elements  # 回傳所有找到的圖片

# ✅ 啟動 Dash 伺服器
if __name__ == "__main__":
    app.run_server(debug=True)


#### 2-2. (接續上)

In [ ]:
    # 收集每張圖片對應的所有 row_index
    for row in table_data:
        image_filename, image_src = get_image_path(row)
        if image_src:
            if image_src not in image_to_rows:
                image_to_rows[image_src] = []
            image_to_rows[image_src].append(row['row_index'])
            # 圖片路徑（image_src）作為鍵，對應的 row_index 會儲存在列表

    # 顯示圖片及其對應的 row_index
    image_elements = []
    for image_src, row_indices in image_to_rows.items():
        image_filename = os.path.basename(image_src)
        image_elements.append(html.Div([
            html.Img(src=image_src, style={"width": "200px", "margin": "10px"}),
            html.P(f"Image Filename: {image_filename}, Row Indices: {', '.join(map(str, row_indices))}")
        ]))

#### 2-3. 表格展示圖片

1. 我們將先收集每張圖片的所有對應 row_index 和檔案名稱。
2. 然後，將這些資料呈現在一個 Dash 表格中，並將圖片顯示在相應的欄位。

In [ ]:
# ✅ 計算 Table 內所有符合條件的圖片，並顯示在 `figure_div`
@app.callback(
    Output("figure_div", "children"),
    Input("table", "data")
)
def update_images(table_data):
    if not table_data:
        return html.Div("無資料")

    # 使用字典來儲存每張圖片對應的 row_index 列表
    image_to_rows = {}

    # 收集每張圖片對應的所有 row_index
    for row in table_data:
        image_filename, image_src = get_image_path(row)
        if image_src:
            if image_src not in image_to_rows:
                image_to_rows[image_src] = []
            image_to_rows[image_src].append(row['row_index'])

    # 準備將圖片顯示在 Dash 表格中
    # 每張圖片資料為一行 (row_index, image_filename, png)加入table_data_for_display中)
    table_data_for_display = []
    for image_src, row_indices in image_to_rows.items():
        image_filename = os.path.basename(image_src)
        row_data = {
            "row_index": ", ".join(map(str, row_indices)),  # 顯示多個對應的 row_index
            "image_filename": image_filename,
            "png": html.Img(src=image_src, style={"width": "100px", "height": "auto"})
        }
        table_data_for_display.append(row_data)

    if not table_data_for_display:
        return html.Div("沒有找到符合條件的圖片")

    return dash_table.DataTable(
        id='image_table',
        columns=[
            {"name": "row_index", "id": "row_index"},
            {"name": "image_filename", "id": "image_filename"},
            {"name": "png", "id": "png"},
        ],
        data=table_data_for_display,
        style_table={'overflowX': 'auto', 'height': '400px'},
        style_cell={'textAlign': 'center'},
        style_data_conditional=[
            {'if': {'column_id': 'png'}, 'width': '120px'}  # 控制圖片的顯示大小
        ]
    )

#### 2-4 設定每個欄位的 高 寬

In [ ]:
        style_data_conditional=[
            # 設定每一欄位的寬度
            {'if': {'column_id': 'row_index'}, 'width': '100px'},
            {'if': {'column_id': 'image_filename'}, 'width': '200px'},
            {'if': {'column_id': 'png'}, 'width': '150px'},  # 限制圖片欄位寬度

            # 設定圖片大小
            {'if': {'column_id': 'png'}, 'height': '120px'},